In [1]:
#!/usr/bin/env python
# coding: utf-8
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras import Input
from keras.initializers import glorot_uniform
from keras.utils import plot_model


def identity_block(x, f_kernel_size, filters, dilation, pad):
    filters_1, filters_2, filters_3 = filters
    x_shortcut = x
    x = Conv2D(filters=filters_1, kernel_size=(1, 1), strides=(1, 1), activation='relu', padding='same',
               kernel_initializer=glorot_uniform(seed=0))(x)
    x = Conv2D(filters=filters_2, kernel_size=f_kernel_size, strides=(1, 1), activation='relu', padding='same',
               dilation_rate=dilation, kernel_initializer=glorot_uniform(seed=0))(x)
    # stage 3
    x = Conv2D(filters=filters_3, kernel_size=(1, 1), strides=(1, 1), padding='same',
               kernel_initializer=glorot_uniform(seed=0))(x)
    # stage 4
    x = Add()([x, x_shortcut])
    x = Activation(activation='relu')(x)

    return x


def conv_block(x, f_kernel_size, filters, strides, dilation, pad):
    filters_1, filters_2, filters_3 = filters
    x_shortcut = x
    # stage 1
    x = Conv2D(filters=filters_1, kernel_size=(1, 1), strides=strides, padding='same', activation='relu',
               kernel_initializer=glorot_uniform(seed=0))(x)
    # stage 2

    x = Conv2D(filters=filters_2, kernel_size=f_kernel_size, strides=(1, 1), activation='relu', padding='same',
               dilation_rate=dilation, kernel_initializer=glorot_uniform(seed=0))(x)
    # stage 3
    x = Conv2D(filters=filters_3, kernel_size=(1, 1), strides=(1, 1), activation='relu', padding='same',
               kernel_initializer=glorot_uniform(seed=0))(x)
    # stage 4
    x_shortcut = Conv2D(filters=filters_3, kernel_size=(1, 1), strides=strides, padding='same',
                        kernel_initializer=glorot_uniform(seed=0))(x_shortcut)
    # stage 5
    x = Add()([x, x_shortcut])
    x = Activation(activation='relu')(x)
    return x


def _resnet101(inputs):
    # 特征图大小下降一半
    x = Conv2D(filters=64, kernel_size=7, strides=2, padding='SAME', activation='relu', use_bias=False)(inputs)
    x = MaxPool2D(pool_size=3, strides=2, padding='SAME')(x)

    x = conv_block(x, f_kernel_size=(3, 3), filters=[64, 64, 256], strides=1, pad=(1, 1), dilation=1)
    x = identity_block(x, f_kernel_size=(3, 3), filters=[64, 64, 256], pad=(1, 1), dilation=1)
    x = identity_block(x, f_kernel_size=(3, 3), filters=[64, 64, 256], pad=(1, 1), dilation=1)

    x = conv_block(x, f_kernel_size=(3, 3), filters=[128, 128, 512], strides=2, pad=(1, 1), dilation=1)
    x = identity_block(x, f_kernel_size=(3, 3), filters=[128, 128, 512], pad=(1, 1), dilation=1)
    x = identity_block(x, f_kernel_size=(3, 3), filters=[128, 128, 512], pad=(1, 1), dilation=1)
    x = identity_block(x, f_kernel_size=(3, 3), filters=[128, 128, 512], pad=(1, 1), dilation=1)

    x = conv_block(x, f_kernel_size=(3, 3), filters=[256, 256, 1024], strides=1, pad=(2, 2), dilation=2)
    for i in range(25):
        x = identity_block(x, f_kernel_size=(3, 3), filters=[256, 256, 1024], pad=(2, 2), dilation=2)

    # x = identity_block(x, f_kernel_size=(3, 3), filters=[256, 256, 1024], pad=(2, 2), dilation=2)
    # x = identity_block(x, f_kernel_size=(3, 3), filters=[256, 256, 1024], pad=(2, 2), dilation=2)
    # x = identity_block(x, f_kernel_size=(3, 3), filters=[256, 256, 1024], pad=(2, 2), dilation=2)
    # x = identity_block(x, f_kernel_size=(3, 3), filters=[256, 256, 1024], pad=(2, 2), dilation=2)
    # x = identity_block(x, f_kernel_size=(3, 3), filters=[256, 256, 1024], pad=(2, 2), dilation=2)

    x = conv_block(x, f_kernel_size=(3, 3), filters=[512, 512, 2048], strides=1, pad=(4, 4), dilation=4)
    x = identity_block(x, f_kernel_size=(3, 3), filters=[256, 256, 2048], pad=(4, 4), dilation=4)
    x = identity_block(x, f_kernel_size=(3, 3), filters=[256, 256, 2048], pad=(4, 4), dilation=4)

    return x

def my_upsampling(x, resize_factor=8, method=0):
    # 0：双线性插值。1：最近邻居法。2：双三次插值法。3：面积插值法
    # NHWC
    img_h = x.shape[1] * resize_factor
    img_w = x.shape[2] * resize_factor
    return tf.image.resize_images(x, (img_h, img_w), 0)


# def ResNet101(input_shape):
#     inputs = Input(input_shape)
#     outputs = _resnet50(inputs)
#
#     model = Model(inputs, outputs)
#
#     return model
def deeplab(input_shape, num_class):
    inputs = Input(input_shape)
    x = _resnet101(inputs)
    # hole = 6
    b1 = Conv2D(filters=1024, kernel_size=3, strides=1, padding='SAME', activation='relu', dilation_rate=6)(x)
    b1 = Conv2D(filters=1024, kernel_size=1, strides=1, activation='relu')(b1)
    b1 = Conv2D(filters=num_class, kernel_size=1, strides=1, activation='relu')(b1)

    # hole = 12
    b2 = Conv2D(filters=1024, kernel_size=3, strides=1, padding='SAME', activation='relu', dilation_rate=12)(x)
    b2 = Conv2D(filters=1024, kernel_size=1, strides=1, activation='relu')(b2)
    b2 = Conv2D(filters=num_class, kernel_size=1, strides=1, activation='relu')(b2)

    # hole = 18
    b3 = Conv2D(filters=1024, kernel_size=3, strides=1, padding='SAME', activation='relu', dilation_rate=18)(x)
    b3 = Conv2D(filters=1024, kernel_size=1, strides=1, activation='relu')(b3)
    b3 = Conv2D(filters=num_class, kernel_size=1, strides=1, activation='relu')(b3)

    # hole = 24
    b4 = Conv2D(filters=1024, kernel_size=3, strides=1, padding='SAME', activation='relu', dilation_rate=24)(x)
    b4 = Conv2D(filters=1024, kernel_size=1, strides=1, activation='relu')(b4)
    b4 = Conv2D(filters=num_class, kernel_size=1, strides=1, activation='relu')(b4)

    s = Add()([b1, b2, b3, b4])
    outputs = Lambda(my_upsampling)(s)

    model = Model(inputs, outputs)

    return model


model = deeplab((512,512,3), 12)
model.summary()

/home/possibleit/anaconda3/envs/cpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/possibleit/anaconda3/envs/cpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/possibleit/anaconda3/envs/cpu/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/possibleit/anaconda3/envs/cpu/lib/pyt

Instructions for updating:
Colocations handled automatically by placer.


Using TensorFlow backend.


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 128, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 128, 64) 4160        max_pooling2d_1[0][0]            
____________________________________________________________________________________________

In [2]:
from generate_data import *
from Unet import *
from generate_data import *
from keras.callbacks import TensorBoard
from keras.models import *
from keras.layers import *
from keras.optimizers import *

In [3]:
model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=["accuracy"])
tb_cb = TensorBoard(log_dir='CamVid/deeplab')
model_checkpoint = keras.callbacks.ModelCheckpoint('CamVid_model_v1.hdf5', monitor='val_loss',verbose=1,save_best_only=True)

In [4]:
train_data = trainGenerator(batch_size=1)
valid_data = validLoad(batch_size=1)
test_data = testGenerator()

In [5]:
history = model.fit_generator(train_data,
                              steps_per_epoch=200,
                              epochs=30,
                              validation_steps=10,
                              validation_data=valid_data,
                              callbacks=[model_checkpoint,tb_cb])

Instructions for updating:
Use tf.cast instead.
Found 367 images belonging to 1 classes.
Found 101 images belonging to 1 classes.
Found 367 images belonging to 1 classes.
Found 101 images belonging to 1 classes.
Epoch 1/30
200/200 [==============================] - 4553s 23s/step - loss: 1.8827 - accuracy: 0.4351 - val_loss: 1.4434 - val_accuracy: 0.5771

Epoch 00001: val_loss improved from inf to 1.44339, saving model to CamVid_model_v1.hdf5
Epoch 2/30
200/200 [==============================] - 4378s 22s/step - loss: 1.2364 - accuracy: 0.6076 - val_loss: 1.2671 - val_accuracy: 0.5913

Epoch 00002: val_loss improved from 1.44339 to 1.26707, saving model to CamVid_model_v1.hdf5
Epoch 3/30
200/200 [==============================] - 4261s 21s/step - loss: 1.1119 - accuracy: 0.6435 - val_loss: 1.1657 - val_accuracy: 0.5930

Epoch 00003: val_loss improved from 1.26707 to 1.16572, saving model to CamVid_model_v1.hdf5
Epoch 4/30
 46/200 [=====>........................] - ETA: 54:23 - loss: 1.

KeyboardInterrupt: 

In [ ]:
model.save_weights('CamVid/deeplab'+'/deeplab_weight.h5')